In [1]:
import numpy
import socket
import cv2
import time
import threading

In [2]:
serverip = socket.gethostbyname(socket.gethostname()) #takes string
stop = False

In [3]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
vidsock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soundsock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

In [4]:
sock.connect((serverip,8888))

vidport = int(sock.recv(50).decode())
vidsock.connect((serverip,vidport))

soundport = int(sock.recv(50).decode())
soundsock.connect((serverip,soundport))
print(soundport,vidport)

61799 61798


In [5]:
stop = False
def send(cam,vidsock):
    cap = cv2.VideoCapture(cam)
    while True:
        try:
            ret,frame = cap.read()
            frame = cv2.resize(frame,(300,300))
            encoded = cv2.imencode(".jpg",frame)[1].tobytes()
            vidsock.send(encoded)
            if stop:
                cap.release()
                print("peer closed the connection")
                sock.close()
                break
        except cv2.error:
            send(cam,vidsock)
        except ConnectionResetError:
            cap.release()

In [ ]:
send(1,vidsock)